In [58]:
import json
import pennylane as qml
import pennylane.numpy as np

edges = [(0, 1), (1, 2), (2, 0), (2, 3)]
num_wires = 4

# We define the Hamiltonian for you!

ops = [qml.PauliZ(0), qml.PauliZ(1),qml.PauliZ(2), qml.PauliZ(3), qml.PauliZ(0)@qml.PauliZ(1), qml.PauliZ(0)@qml.PauliZ(2),qml.PauliZ(1)@qml.PauliZ(2),qml.PauliZ(2)@qml.PauliZ(3)]
coeffs = [0.5, 0.5, 1.25, -0.25, 0.75, 0.75, 0.75, 0.75]

cost_hamiltonian = qml.Hamiltonian(coeffs, ops)

# Write any helper functions you need here
true_value = np.min(np.linalg.eigvalsh(cost_hamiltonian.sparse_matrix().todense()))

dev = qml.device('default.mixed', wires = num_wires)

@qml.qnode(dev) 
def qaoa_circuit(gammabeta, noise_param):

    for i in range(num_wires):
        qml.RY(np.pi/2, i)

    for param in gammabeta: 
        gamma, alpha = param
        gamma /= 2
        alpha /= 2
        qml.RZ(2*gamma*0.5, wires=0)
        qml.RZ(2*gamma*0.5, wires=1)
        qml.RZ(2*gamma*1.25, wires=2)
        qml.RZ(2*gamma*-0.25, wires=3)

        qml.CNOT(wires=[0,1])
        # qml.DepolarizingChannel(noise_param, wires=1)

        qml.RZ(2*gamma*0.75, 1)
        qml.CNOT(wires=[0,1])
        qml.DepolarizingChannel(noise_param, wires=1)

        qml.CNOT(wires=[0,2])
        # qml.DepolarizingChannel(noise_param, wires=2)

        qml.RZ(2*gamma*0.75, 2)
        qml.CNOT(wires=[0,2])
        # qml.DepolarizingChannel(noise_param, wires=2)

        qml.CNOT(wires=[1,2])
        # qml.DepolarizingChannel(noise_param, wires=2)

        qml.RZ(2*gamma*0.75, 2)
        qml.CNOT(wires=[1,2])    
        qml.DepolarizingChannel(noise_param, wires=2)

        qml.CNOT(wires=[2,3])
        qml.RZ(2*gamma*0.75, 3)
        qml.CNOT(wires=[2,3])                    
        qml.DepolarizingChannel(noise_param, wires=3)

        for i in range(num_wires):
            qml.RX(2*alpha, wires=i)      

        return qml.expval(cost_hamiltonian)

def approximation_ratio(qaoa_depth, noise_param):
    params = np.array([[0.5, 0.5]], requires_grad=True)
    np.repeat(params,qaoa_depth,axis=0)    
    print ("approximation_ratio : ", qaoa_depth, noise_param, params)
    
    def cost_function(p):
        return qaoa_circuit(np.repeat(p, qaoa_depth, axis=0), noise_param)
    
    opt = qml.AdamOptimizer(stepsize=0.1)

    for it in range(500):
        params, cost = opt.step_and_cost(cost_function, params)
        if it % 50 == 0:
            print (it, " iter, cost = ", cost, cost/true_value, params)

    # qml.draw_mpl(qaoa_circuit, decimals=2)(params, 0)

    minval = cost_function(params)
    return minval/true_value

random_params = np.array([np.random.rand(2)])

ops_2 = [qml.PauliX(0), qml.PauliX(1), qml.PauliX(2), qml.PauliX(3)]
coeffs_2 = [1,1,1,1]

mixer_hamiltonian = qml.Hamiltonian(coeffs_2, ops_2)

@qml.qnode(dev)
def noiseless_qaoa(params):
    for wire in range(num_wires):
        qml.Hadamard(wires = wire)

    for elem in params:
        qml.ApproxTimeEvolution(cost_hamiltonian, elem[0], 1)
        qml.ApproxTimeEvolution(mixer_hamiltonian, elem[1],1)

    return qml.expval(cost_hamiltonian)

random_params = np.array([np.random.rand(2)])
circuit_check = (np.isclose(noiseless_qaoa(random_params) - qaoa_circuit(random_params,0),0)).numpy()

def run(test_case_input: str) -> str:
    input = json.loads(test_case_input)

    output = approximation_ratio(*input)
    print ("Run output === ", output)
    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    print (solution_output, expected_output)
    tape = qaoa_circuit.qtape
    names = [op.name for op in tape.operations]
    random_params = np.array([np.random.rand(2)])
    assert circuit_check, "qaoa_circuit is not doing what it's expected to."

    assert names.count('ApproxTimeEvolution') == 0, "Your circuit must not use the built-in PennyLane Trotterization."
     
    assert set(names) == {'DepolarizingChannel', 'RX', 'RY', 'RZ', 'CNOT'}, "Your circuit must use qml.RX, qml.RY, qml.RZ, qml.CNOT, and qml.DepolarizingChannel."

    assert solution_output > expected_output - 0.02


# These are the public test cases
test_cases = [
    ('[1, 0.003]', '0.1307'),
    ('[2,0.005]', '0.4875'),
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[1, 0.003]'...
approximation_ratio :  1 0.003 [[0.5 0.5]]
0  iter, cost =  1.2622665707284622 -0.42075552357615403 [[0.40000001 0.4       ]]
50  iter, cost =  -1.328582992615598 0.4428609975385327 [[-0.70586321  0.91675101]]
100  iter, cost =  -1.4036017427635081 0.46786724758783604 [[-0.67505169  0.79565453]]
150  iter, cost =  -1.4043856161373511 0.46812853871245036 [[-0.69532114  0.79469475]]
200  iter, cost =  -1.4043882787939714 0.46812942626465714 [[-0.69509566  0.79423977]]
250  iter, cost =  -1.4043883016193837 0.4681294338731279 [[-0.69520925  0.79425241]]
300  iter, cost =  -1.4043883017077157 0.4681294339025719 [[-0.69521148  0.79424741]]


KeyboardInterrupt: 

In [37]:
features = np.array([0.1, 0.2], requires_grad=True)
features

tensor([0.1, 0.2], requires_grad=True)

In [61]:
# passed 1 teest

edges = [(0, 1), (1, 2), (2, 0), (2, 3)]
num_wires = 4

# We define the Hamiltonian for you!

ops = [qml.PauliZ(0), qml.PauliZ(1),qml.PauliZ(2), qml.PauliZ(3), qml.PauliZ(0)@qml.PauliZ(1), qml.PauliZ(0)@qml.PauliZ(2),qml.PauliZ(1)@qml.PauliZ(2),qml.PauliZ(2)@qml.PauliZ(3)]
coeffs = [0.5, 0.5, 1.25, -0.25, 0.75, 0.75, 0.75, 0.75]

cost_hamiltonian = qml.Hamiltonian(coeffs, ops)

# Write any helper functions you need here
true_value = np.min(np.linalg.eigvalsh(cost_hamiltonian.sparse_matrix().todense()))

dev = qml.device('default.mixed', wires = num_wires)

@qml.qnode(dev) 
def qaoa_circuit(params, noise_param):

    """
    Define the noisy QAOA circuit with only CNOT and rotation gates, with Depolarizing noise
    in the target qubit of each CNOT gate.

    Args:
        params(list(list(float))): A list with length equal to the QAOA depth. Each element is a list that contains 
        the two QAOA parameters of each layer.
        noise_param (float): The noise parameter associated with the depolarization gate

    Returns: 
        (np.tensor): A numpy tensor of 1 element corresponding to the expectation value of the cost Hamiltonian
    
    """

    for i in range(num_wires):
        qml.RY(np.pi/2, i)
        # qml.Hadamard(i)
    for param in params: 
        gamma, alpha = param 

        qml.RZ(2*gamma*0.5, wires=0)
        qml.RZ(2*gamma*0.5, wires=1)
        qml.RZ(2*gamma*1.25, wires=2)
        qml.RZ(2*gamma*-0.25, wires=3)

        qml.CNOT(wires=[0,1])
        qml.DepolarizingChannel(noise_param, wires=1)
        qml.RZ(2*gamma*0.75, 1)
        qml.CNOT(wires=[0,1])
        qml.DepolarizingChannel(noise_param, wires=1)

        qml.CNOT(wires=[0,2])
        qml.DepolarizingChannel(noise_param, wires=2)
        qml.RZ(2*gamma*0.75, 2)
        qml.CNOT(wires=[0,2])
        qml.DepolarizingChannel(noise_param, wires=2)

        qml.CNOT(wires=[1,2])
        qml.DepolarizingChannel(noise_param, wires=2)
        qml.RZ(2*gamma*0.75, 2)
        qml.CNOT(wires=[1,2])    
        qml.DepolarizingChannel(noise_param, wires=2)

        qml.CNOT(wires=[2,3])
        qml.DepolarizingChannel(noise_param, wires=3)
        qml.RZ(2*gamma*0.75, 3)
        qml.CNOT(wires=[2,3])                    
        qml.DepolarizingChannel(noise_param, wires=3)

        for i in range(num_wires):
            qml.RX(2*alpha, wires=i)      
            
        return qml.expval(cost_hamiltonian)
def approximation_ratio(qaoa_depth, noise_param):
    """
    Returns the approximation ratio of the QAOA algorithm for the Minimum Vertex Cover of the given graph
    with depolarizing gates after each native CNOT gate

    Args:
        qaoa_depth (float): The number of cost/mixer layer in the QAOA algorithm used
        noise_param (float): The noise parameter associated with the depolarization gate
    
    Returns: 
        (float): The approximation ratio for the noisy QAOA
    """

    print ("approximation_ratio : ", qaoa_depth, noise_param)
    params = np.random.uniform(0, 2*np.pi, (qaoa_depth, 2))

    def cost_function(p):
        return qaoa_circuit(p, noise_param)
    
    opt = qml.AdamOptimizer(stepsize=0.1)

    for it in range(500):
        params, cost = opt.step_and_cost(cost_function, params)
        if it % 50 == 0:
            print (it, " iter, cost = ", cost, cost/true_value)

    minval = cost_function(params)
    return minval/true_value
# These functions are responsible for testing the solution.
random_params = np.array([np.random.rand(2)])

ops_2 = [qml.PauliX(0), qml.PauliX(1), qml.PauliX(2), qml.PauliX(3)]
coeffs_2 = [1,1,1,1]

mixer_hamiltonian = qml.Hamiltonian(coeffs_2, ops_2)

@qml.qnode(dev)
def noiseless_qaoa(params):

    for wire in range(num_wires):

        qml.Hadamard(wires = wire)

    for elem in params:

        qml.ApproxTimeEvolution(cost_hamiltonian, elem[0], 1)
        qml.ApproxTimeEvolution(mixer_hamiltonian, elem[1],1)

    return qml.expval(cost_hamiltonian)

random_params = np.array([np.random.rand(2)])

circuit_check = (np.isclose(noiseless_qaoa(random_params) - qaoa_circuit(random_params,0),0)).numpy()

def run(test_case_input: str) -> str:
    input = json.loads(test_case_input)
    output = approximation_ratio(*input)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    
    tape = qaoa_circuit.qtape
    names = [op.name for op in tape.operations]
    random_params = np.array([np.random.rand(2)])

    assert circuit_check, "qaoa_circuit is not doing what it's expected to."

    assert names.count('ApproxTimeEvolution') == 0, "Your circuit must not use the built-in PennyLane Trotterization."
     
    assert set(names) == {'DepolarizingChannel', 'RX', 'RY', 'RZ', 'CNOT'}, "Your circuit must use qml.RX, qml.RY, qml.RZ, qml.CNOT, and qml.DepolarizingChannel."

    assert solution_output > expected_output - 0.02

# These are the public test cases
test_cases = [
    ('[1, 0.003]', '0.1307'),
    ('[2,0.005]', '0.4875'),
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")    

Running test case 0 with input '[1, 0.003]'...
approximation_ratio :  1 0.003
0  iter, cost =  -0.1150105740655393 0.038336858021846436
50  iter, cost =  -0.39119404791244666 0.13039801597081555
100  iter, cost =  -0.3920527933504877 0.13068426445016257
150  iter, cost =  -0.39205784454789205 0.13068594818263068
200  iter, cost =  -0.3920578650105109 0.13068595500350363
250  iter, cost =  -0.39205786517202706 0.13068595505734235
300  iter, cost =  -0.39205786517352686 0.13068595505784228
350  iter, cost =  -0.3920578651735358 0.13068595505784528
400  iter, cost =  -0.3920578651735358 0.13068595505784528
450  iter, cost =  -0.3920578651735356 0.1306859550578452
Correct!
Running test case 1 with input '[2,0.005]'...
approximation_ratio :  2 0.005
0  iter, cost =  0.30233483737013844 -0.10077827912337949
50  iter, cost =  -0.3871767850573692 0.1290589283524564
100  iter, cost =  -0.3893966616422878 0.12979888721409594
150  iter, cost =  -0.389403363485609 0.12980112116186968
200  iter, co

AssertionError: 